<a href="https://colab.research.google.com/github/mbc2009/Lmp_ML/blob/main/Model_trainning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Enviornment Initialization

In [87]:
%%capture
%%bash

# remove unnecessary
rm -rf *

# update pip
python -m pip install --upgrade pip

# install package
pip install opencv-python pillow
pip install segmentation_models_pytorch
pip install -q kaggle
pip install dropbox
pip install scikit-image

In [88]:
# basic import
import  os, sys, time, math, random, math, psutil
from    typing                  import  List, Tuple
from    dropbox                 import  Dropbox
from    tqdm                    import  tqdm
from    matplotlib              import  pyplot      as plt
import  numpy                                       as np
import  pandas                                      as pd
import  zipfile
import  warnings
import  shutil

from    skimage                 import  io
from    scipy                   import  interpolate
from    scipy.interpolate       import  RegularGridInterpolator
from    scipy.ndimage           import  generic_filter
from    PIL                     import  Image

import  torch
from    torch                   import  nn
from    torch.nn                import  functional  as F
import  torch.optim                                 as optim
import  torchvision.transforms.functional           as TF
from    torch.utils.data        import  Dataset, DataLoader, TensorDataset, random_split, Subset
from    torchvision             import  transforms, models
from    torchvision.transforms  import  *
import  kagglehub

# 1. Configuration

In [89]:
class Confuration():
  # define paths to directory
  home_dir          = os.path.expanduser("~")
  working_dir       = os.path.join(os.getcwd(),'Lmp_ML')
  DataBase_dir      = os.path.join(working_dir,'DataBase')
  kaggle_config_dir = os.path.join(home_dir, ".kaggle")
  kaggle_json_dir   = os.path.join(working_dir, "kaggle.json")

conf = Confuration()

In [90]:
# import code from git hub
!git clone https://github.com/mbc2009/Lmp_ML

Cloning into 'Lmp_ML'...
remote: Enumerating objects: 503, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 503 (delta 19), reused 11 (delta 4), pack-reused 450 (from 3)
Receiving objects: 100% (503/503), 128.59 MiB | 8.27 MiB/s, done.
Resolving deltas: 100% (235/235), done.


In [91]:
# make directory
os.makedirs(conf.kaggle_config_dir, exist_ok=True)
os.makedirs(conf.DataBase_dir, exist_ok=True)

In [92]:


# download
original_path = kagglehub.dataset_download("mbc2009/heat-and-mass-transfer")
shutil.move(original_path, conf.DataBase_dir)

100%|██████████| 773M/773M [00:15<00:00, 53.5MB/s]

Extracting files...


'/content/Lmp_ML/DataBase/4'